In [ ]:
# Essential imports for digital watermarking
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data, color, filters, morphology
from scipy import fftpack, ndimage
import hashlib
import random
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('seaborn-v0_8')

print("🔐 Digital Watermarking Systems Ready!")
print("🎯 Protecting intellectual property through invisible embedding")

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)


In [ ]:
# Load test images for watermarking experiments
def load_test_images():
    """Load various test images for watermarking"""
    images = {
        'Lena': data.camera(),           # Standard test image
        'Coins': data.coins(),           # High detail image
        'Astronaut': data.astronaut(),   # Color image
        'Text': data.text(),             # Text-heavy image
        'Coffee': data.coffee()          # Natural scene
    }
    
    # Convert color images to grayscale for initial experiments
    for name in images:
        if len(images[name].shape) == 3:
            images[name] = color.rgb2gray(images[name])
            images[name] = (images[name] * 255).astype(np.uint8)
    
    return images

def analyze_image_characteristics(images):
    """Analyze images for watermarking suitability"""
    
    characteristics = {}
    
    for name, img in images.items():
        # Calculate image statistics
        mean_intensity = np.mean(img)
        std_intensity = np.std(img)
        
        # Edge content (measure of detail)
        edges = filters.sobel(img)
        edge_content = np.mean(edges)
        
        # Texture measure (local standard deviation)
        texture = filters.rank.standard(img, morphology.disk(3))
        texture_measure = np.mean(texture)
        
        # Frequency content
        f_transform = fftpack.fft2(img)
        f_spectrum = np.abs(f_transform)
        high_freq_energy = np.sum(f_spectrum[f_spectrum.shape[0]//4:, f_spectrum.shape[1]//4:])
        total_energy = np.sum(f_spectrum)
        high_freq_ratio = high_freq_energy / total_energy
        
        characteristics[name] = {
            'mean': mean_intensity,
            'std': std_intensity,
            'edge_content': edge_content,
            'texture': texture_measure,
            'high_freq_ratio': high_freq_ratio
        }
    
    return characteristics

# Load test images
test_images = load_test_images()
image_chars = analyze_image_characteristics(test_images)

# Visualize test images and their characteristics
fig, axes = plt.subplots(2, len(test_images), figsize=(20, 8))
fig.suptitle('Test Images for Watermarking Analysis', fontsize=16, fontweight='bold')

for idx, (name, img) in enumerate(test_images.items()):
    # Original image
    axes[0, idx].imshow(img, cmap='gray')
    axes[0, idx].set_title(f'{name}')
    axes[0, idx].axis('off')
    
    # Edge map
    edges = filters.sobel(img)
    axes[1, idx].imshow(edges, cmap='hot')
    axes[1, idx].set_title(f'{name} - Edge Content')
    axes[1, idx].axis('off')

plt.tight_layout()
plt.show()

# Print image characteristics
print("📊 Image Characteristics for Watermarking:")
print("=" * 100)
print(f"{'Image':<12} | {'Mean':<8} | {'Std':<8} | {'Edge':<8} | {'Texture':<8} | {'HF Ratio':<8}")
print("-" * 100)

for name, chars in image_chars.items():
    print(f"{name:<12} | {chars['mean']:<8.1f} | {chars['std']:<8.1f} | "
          f"{chars['edge_content']:<8.3f} | {chars['texture']:<8.1f} | {chars['high_freq_ratio']:<8.3f}")

print("\\n🔍 Watermarking Insights:")
print("• High texture images can hide more watermark information")
print("• High frequency content provides better embedding opportunities")
print("• Edge-rich regions are more suitable for robust watermarking")
print("• Smooth regions require careful embedding to maintain imperceptibility")


In [ ]:
# Visible Watermarking Implementation
class VisibleWatermarking:
    """Comprehensive visible watermarking system"""
    
    def __init__(self):
        self.methods = {
            'alpha_blend': self._alpha_blend,
            'adaptive': self._adaptive_watermark,
            'texture_based': self._texture_based,
            'corner_logo': self._corner_logo
        }
    
    def _alpha_blend(self, image, watermark, alpha=0.3):
        """Simple alpha blending watermark"""
        # Resize watermark to match image size
        h, w = image.shape
        watermark_resized = cv2.resize(watermark, (w, h))
        
        # Apply alpha blending
        watermarked = alpha * watermark_resized + (1 - alpha) * image
        return watermarked.astype(np.uint8)
    
    def _adaptive_watermark(self, image, watermark, base_alpha=0.2):
        """Adaptive watermark based on local image content"""
        h, w = image.shape
        watermark_resized = cv2.resize(watermark, (w, h))
        
        # Calculate local variance to determine watermark strength
        local_variance = filters.rank.variance(image, morphology.disk(5))
        
        # Normalize variance to [0, 1]
        variance_norm = (local_variance - local_variance.min()) / (local_variance.max() - local_variance.min())
        
        # Higher variance areas get stronger watermark
        adaptive_alpha = base_alpha + 0.3 * variance_norm
        adaptive_alpha = np.clip(adaptive_alpha, 0, 0.8)
        
        # Apply adaptive blending
        watermarked = adaptive_alpha * watermark_resized + (1 - adaptive_alpha) * image
        return watermarked.astype(np.uint8)
    
    def _texture_based(self, image, watermark, strength=0.4):
        """Texture-based watermark embedding"""
        h, w = image.shape
        watermark_resized = cv2.resize(watermark, (w, h))
        
        # Detect texture regions using Gabor filters
        texture_response = 0
        for theta in [0, 45, 90, 135]:
            real, _ = filters.gabor(image, frequency=0.6, theta=np.radians(theta))
            texture_response += np.abs(real)
        
        # Normalize texture response
        texture_norm = (texture_response - texture_response.min()) / (texture_response.max() - texture_response.min())
        
        # Embed watermark more strongly in textured regions
        texture_alpha = strength * texture_norm
        texture_alpha = np.clip(texture_alpha, 0, 0.7)
        
        watermarked = texture_alpha * watermark_resized + (1 - texture_alpha) * image
        return watermarked.astype(np.uint8)
    
    def _corner_logo(self, image, watermark, position='bottom_right', size_ratio=0.2, alpha=0.6):
        """Corner logo watermark"""
        h, w = image.shape
        logo_size = int(min(h, w) * size_ratio)
        
        # Resize watermark to logo size
        logo = cv2.resize(watermark, (logo_size, logo_size))
        
        # Calculate position
        if position == 'bottom_right':
            start_y, start_x = h - logo_size - 10, w - logo_size - 10
        elif position == 'top_left':
            start_y, start_x = 10, 10
        elif position == 'top_right':
            start_y, start_x = 10, w - logo_size - 10
        else:  # bottom_left
            start_y, start_x = h - logo_size - 10, 10
        
        # Create watermarked image
        watermarked = image.copy().astype(np.float64)
        end_y, end_x = start_y + logo_size, start_x + logo_size
        
        # Apply alpha blending to logo region
        watermarked[start_y:end_y, start_x:end_x] = (alpha * logo + 
                                                     (1 - alpha) * image[start_y:end_y, start_x:end_x])
        
        return watermarked.astype(np.uint8)
    
    def create_text_watermark(self, text, size=(200, 200), font_size=20):
        """Create text watermark"""
        # Create image with text
        img = Image.new('L', size, 0)  # Black background
        draw = ImageDraw.Draw(img)
        
        try:
            # Try to use a better font
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            # Fallback to default font
            font = ImageFont.load_default()
        
        # Get text size and center it
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        
        x = (size[0] - text_width) // 2
        y = (size[1] - text_height) // 2
        
        draw.text((x, y), text, fill=255, font=font)
        
        return np.array(img)
    
    def apply_watermark(self, image, watermark, method='alpha_blend', **kwargs):
        """Apply watermark using specified method"""
        if method not in self.methods:
            raise ValueError(f"Unknown method: {method}")
        
        return self.methods[method](image, watermark, **kwargs)

# Create watermark system
watermark_system = VisibleWatermarking()

# Create different types of watermarks
text_watermark = watermark_system.create_text_watermark("© 2025 AI Intern", (300, 300), 25)
logo_watermark = np.zeros((100, 100))
cv2.circle(logo_watermark, (50, 50), 40, 255, -1)  # Simple circular logo
cv2.circle(logo_watermark, (50, 50), 30, 0, -1)    # Inner circle

# Test visible watermarking on sample image
test_image = test_images['Lena']

# Apply different visible watermarking methods
visible_results = {}
methods_to_test = [
    ('Alpha Blend', 'alpha_blend', {'alpha': 0.3}),
    ('Adaptive', 'adaptive', {'base_alpha': 0.2}),
    ('Texture Based', 'texture_based', {'strength': 0.4}),
    ('Corner Logo', 'corner_logo', {'alpha': 0.6, 'size_ratio': 0.15})
]

for name, method, params in methods_to_test:
    if method == 'corner_logo':
        watermarked = watermark_system.apply_watermark(test_image, logo_watermark, method, **params)
    else:
        watermarked = watermark_system.apply_watermark(test_image, text_watermark, method, **params)
    visible_results[name] = watermarked

# Visualize visible watermarking results
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Visible Watermarking Techniques', fontsize=16, fontweight='bold')

# Show original image
axes[0, 0].imshow(test_image, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

# Show watermarks
axes[0, 1].imshow(text_watermark, cmap='gray')
axes[0, 1].set_title('Text Watermark')
axes[0, 1].axis('off')

axes[0, 2].imshow(logo_watermark, cmap='gray')
axes[0, 2].set_title('Logo Watermark')
axes[0, 2].axis('off')

# Show watermarked results
for idx, (name, result) in enumerate(visible_results.items()):
    row = 1
    col = idx % 3
    if idx < 3:
        axes[row, col].imshow(result, cmap='gray')
        axes[row, col].set_title(name)
        axes[row, col].axis('off')

# Remove empty subplot if any
if len(visible_results) < 3:
    axes[1, 2].remove()

plt.tight_layout()
plt.show()

print("👁️ Visible Watermarking Analysis:")
print("• Alpha Blend: Simple but may overpower image details")
print("• Adaptive: Adjusts to local image content for better integration")
print("• Texture Based: Leverages image texture for natural appearance")
print("• Corner Logo: Professional branding without content obstruction")


In [ ]:
# Invisible Watermarking Implementation
class InvisibleWatermarking:
    """Comprehensive invisible watermarking system"""
    
    def __init__(self):
        self.embedding_methods = {
            'lsb': self._lsb_embed,
            'dct': self._dct_embed,
            'dft': self._dft_embed,
            'additive': self._additive_embed
        }
        
        self.extraction_methods = {
            'lsb': self._lsb_extract,
            'dct': self._dct_extract,
            'dft': self._dft_extract,
            'additive': self._additive_extract
        }
    
    def _lsb_embed(self, image, watermark_bits, num_bits=1):
        """LSB embedding method"""
        watermarked = image.copy()
        flat_image = watermarked.flatten()
        
        # Create bit mask
        mask = (0xFF << num_bits) & 0xFF
        
        # Embed watermark bits
        for i, bit in enumerate(watermark_bits):
            if i >= len(flat_image):
                break
            
            # Clear LSBs and set new value
            flat_image[i] = (flat_image[i] & mask) | bit
        
        return flat_image.reshape(image.shape)
    
    def _lsb_extract(self, watermarked_image, num_bits=1, watermark_length=None):
        """LSB extraction method"""
        flat_image = watermarked_image.flatten()
        
        # Extract LSBs
        mask = (1 << num_bits) - 1
        extracted_bits = []
        
        for i in range(len(flat_image)):
            if watermark_length and len(extracted_bits) >= watermark_length:
                break
            extracted_bits.append(flat_image[i] & mask)
        
        return np.array(extracted_bits)
    
    def _dct_embed(self, image, watermark, alpha=0.1):
        """DCT-based embedding"""
        # Apply DCT
        dct_image = fftpack.dctn(image.astype(np.float64), norm='ortho')
        
        # Resize watermark to match image size
        h, w = image.shape
        watermark_resized = cv2.resize(watermark.astype(np.float64), (w, h))
        
        # Embed in mid-frequency coefficients
        dct_watermarked = dct_image + alpha * watermark_resized
        
        # Apply inverse DCT
        watermarked = fftpack.idctn(dct_watermarked, norm='ortho')
        
        return np.clip(watermarked, 0, 255).astype(np.uint8)
    
    def _dct_extract(self, watermarked_image, original_image, alpha=0.1):
        """DCT-based extraction"""
        # Apply DCT to both images
        dct_watermarked = fftpack.dctn(watermarked_image.astype(np.float64), norm='ortho')
        dct_original = fftpack.dctn(original_image.astype(np.float64), norm='ortho')
        
        # Extract watermark
        extracted_watermark = (dct_watermarked - dct_original) / alpha
        
        return extracted_watermark
    
    def _dft_embed(self, image, watermark, alpha=0.1):
        """DFT-based embedding"""
        # Apply 2D FFT
        fft_image = fftpack.fft2(image.astype(np.float64))
        
        # Resize watermark
        h, w = image.shape
        watermark_resized = cv2.resize(watermark.astype(np.float64), (w, h))
        
        # Embed in frequency domain
        fft_watermarked = fft_image + alpha * watermark_resized
        
        # Apply inverse FFT
        watermarked = np.real(fftpack.ifft2(fft_watermarked))
        
        return np.clip(watermarked, 0, 255).astype(np.uint8)
    
    def _dft_extract(self, watermarked_image, original_image, alpha=0.1):
        """DFT-based extraction"""
        # Apply FFT to both images
        fft_watermarked = fftpack.fft2(watermarked_image.astype(np.float64))
        fft_original = fftpack.fft2(original_image.astype(np.float64))
        
        # Extract watermark
        extracted_watermark = np.real((fft_watermarked - fft_original) / alpha)
        
        return extracted_watermark
    
    def _additive_embed(self, image, watermark, alpha=0.1):
        """Simple additive embedding"""
        h, w = image.shape
        watermark_resized = cv2.resize(watermark.astype(np.float64), (w, h))
        
        # Add watermark to image
        watermarked = image.astype(np.float64) + alpha * watermark_resized
        
        return np.clip(watermarked, 0, 255).astype(np.uint8)
    
    def _additive_extract(self, watermarked_image, original_image, alpha=0.1):
        """Simple additive extraction"""
        extracted_watermark = (watermarked_image.astype(np.float64) - 
                              original_image.astype(np.float64)) / alpha
        
        return extracted_watermark
    
    def embed_watermark(self, image, watermark, method='dct', **kwargs):
        """Embed watermark using specified method"""
        if method not in self.embedding_methods:
            raise ValueError(f"Unknown embedding method: {method}")
        
        return self.embedding_methods[method](image, watermark, **kwargs)
    
    def extract_watermark(self, watermarked_image, original_image=None, method='dct', **kwargs):
        """Extract watermark using specified method"""
        if method not in self.extraction_methods:
            raise ValueError(f"Unknown extraction method: {method}")
        
        if method in ['dct', 'dft', 'additive'] and original_image is None:
            raise ValueError(f"Method {method} requires original image for extraction")
        
        if method == 'lsb':
            return self.extraction_methods[method](watermarked_image, **kwargs)
        else:
            return self.extraction_methods[method](watermarked_image, original_image, **kwargs)

def calculate_psnr(original, watermarked):
    """Calculate Peak Signal-to-Noise Ratio"""
    mse = np.mean((original - watermarked) ** 2)
    if mse == 0:
        return float('inf')
    
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_correlation(watermark1, watermark2):
    """Calculate normalized correlation between two watermarks"""
    # Flatten and normalize
    w1_flat = watermark1.flatten()
    w2_flat = watermark2.flatten()
    
    # Calculate correlation coefficient
    correlation = np.corrcoef(w1_flat, w2_flat)[0, 1]
    return correlation

# Create invisible watermarking system
invisible_system = InvisibleWatermarking()

# Create binary watermark for testing
def create_binary_watermark(size=(64, 64), pattern='checkerboard'):
    """Create binary watermark patterns"""
    h, w = size
    
    if pattern == 'checkerboard':
        watermark = np.zeros((h, w))
        for i in range(h):
            for j in range(w):
                if (i // 8 + j // 8) % 2 == 0:
                    watermark[i, j] = 255
    
    elif pattern == 'logo':
        watermark = np.zeros((h, w))
        center_x, center_y = w // 2, h // 2
        radius = min(h, w) // 4
        
        # Create circular logo
        for i in range(h):
            for j in range(w):
                if (i - center_y)**2 + (j - center_x)**2 <= radius**2:
                    watermark[i, j] = 255
    
    elif pattern == 'text':
        watermark = np.zeros((h, w))
        # Simple text pattern (AI)
        watermark[h//4:3*h//4, w//4:w//2] = 255  # A
        watermark[h//4:3*h//4, 3*w//4:w-w//8] = 255  # I
    
    return watermark

# Test invisible watermarking
test_image = test_images['Lena']
binary_watermark = create_binary_watermark((64, 64), 'logo')

# Test different invisible methods
invisible_methods = ['lsb', 'dct', 'dft', 'additive']
invisible_results = {}
extraction_results = {}
quality_metrics = {}

print("🔒 Testing Invisible Watermarking Methods...")

for method in invisible_methods:
    print(f"  Processing {method.upper()}...")
    
    try:
        if method == 'lsb':
            # For LSB, convert watermark to bits
            watermark_bits = (binary_watermark.flatten() > 128).astype(np.uint8)
            watermarked = invisible_system.embed_watermark(test_image, watermark_bits, method=method)
            extracted = invisible_system.extract_watermark(watermarked, method=method, 
                                                         num_bits=1, watermark_length=len(watermark_bits))
            extracted_watermark = (extracted.reshape(binary_watermark.shape) * 255).astype(np.uint8)
        else:
            watermarked = invisible_system.embed_watermark(test_image, binary_watermark, method=method, alpha=0.1)
            extracted = invisible_system.extract_watermark(watermarked, test_image, method=method, alpha=0.1)
            extracted_watermark = np.clip(extracted, 0, 255).astype(np.uint8)
        
        # Calculate quality metrics
        psnr = calculate_psnr(test_image.astype(np.float64), watermarked.astype(np.float64))
        correlation = calculate_correlation(binary_watermark, extracted_watermark)
        
        invisible_results[method] = watermarked
        extraction_results[method] = extracted_watermark
        quality_metrics[method] = {'psnr': psnr, 'correlation': correlation}
        
    except Exception as e:
        print(f"    Error with {method}: {e}")
        continue

# Visualize invisible watermarking results
fig, axes = plt.subplots(3, len(invisible_results) + 1, figsize=(20, 12))
fig.suptitle('Invisible Watermarking: Embedding and Extraction', fontsize=16, fontweight='bold')

# First column: originals
axes[0, 0].imshow(test_image, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[1, 0].imshow(binary_watermark, cmap='gray')
axes[1, 0].set_title('Original Watermark')
axes[1, 0].axis('off')

axes[2, 0].text(0.5, 0.5, 'Quality Metrics', ha='center', va='center', 
                transform=axes[2, 0].transAxes, fontsize=12, fontweight='bold')
axes[2, 0].axis('off')

# Results for each method
for idx, (method, watermarked) in enumerate(invisible_results.items()):
    col = idx + 1
    
    # Watermarked image
    axes[0, col].imshow(watermarked, cmap='gray')
    axes[0, col].set_title(f'{method.upper()} Watermarked')
    axes[0, col].axis('off')
    
    # Extracted watermark
    axes[1, col].imshow(extraction_results[method], cmap='gray')
    axes[1, col].set_title(f'{method.upper()} Extracted')
    axes[1, col].axis('off')
    
    # Quality metrics
    metrics = quality_metrics[method]
    metrics_text = f"PSNR: {metrics['psnr']:.2f} dB\\nCorr: {metrics['correlation']:.3f}"
    axes[2, col].text(0.5, 0.5, metrics_text, ha='center', va='center',
                     transform=axes[2, col].transAxes, fontsize=10)
    axes[2, col].axis('off')

plt.tight_layout()
plt.show()

# Print quality comparison
print("\\n📊 Invisible Watermarking Quality Analysis:")
print("=" * 60)
print(f"{'Method':<12} | {'PSNR (dB)':<10} | {'Correlation':<12} | {'Quality':<10}")
print("-" * 60)

for method, metrics in quality_metrics.items():
    psnr = metrics['psnr']
    corr = metrics['correlation']
    
    # Determine quality rating
    if psnr > 40 and corr > 0.8:
        quality = "Excellent"
    elif psnr > 30 and corr > 0.6:
        quality = "Good"
    elif psnr > 25 and corr > 0.4:
        quality = "Fair"
    else:
        quality = "Poor"
    
    print(f"{method.upper():<12} | {psnr:<10.2f} | {corr:<12.3f} | {quality:<10}")

print("\\n🔍 Method Analysis:")
print("• LSB: High capacity but fragile to compression")
print("• DCT: Good balance of imperceptibility and robustness")
print("• DFT: Global frequency embedding, moderate robustness")
print("• Additive: Simple but limited robustness")
